In [2]:
import scipy.sparse
import numpy as np

n=10 # number of features
m=20000 # number of training examples

X = scipy.sparse.random(m,n, density=.2).toarray() # Guarantees sparse grad updates
real_w = np.random.uniform(0,1,size=(n,1)) # Define our true weight vector

X = X/X.max() # Normalizing for training

y = np.dot(X,real_w)

In [7]:
from multiprocessing.sharedctypes import Array
from ctypes import c_double
import numpy as np

coef_shared = Array(c_double, 
        (np.random.normal(size=(n,1)) * 1./np.sqrt(n)).flat,
        lock=False) # Hogwild!
w = np.frombuffer(coef_shared)
w = w.reshape((n,1)) 

In [11]:
learning_rate = .001
def mse_gradient_step(X_y_tuple):
    global w # Only for instructive purposes!
    X, y = X_y_tuple # Required for how multiprocessing.Pool.map works
    
    # Calculate the gradient
    err = y.reshape((len(y),1))-np.dot(X,w)
    grad = -2.*np.dot(np.transpose(X),err)/ X.shape[0]

    # Update the nonzero weights one at a time
    for index in np.where(abs(grad) > .01)[0]:
        coef_shared[index] -= learning_rate*grad[index,0]


In [12]:
batch_size=1
examples=[None]*int(X.shape[0]/float(batch_size))
for k in range(int(X.shape[0]/float(batch_size))):
    Xx = X[k*batch_size : (k+1)*batch_size,:].reshape((batch_size,X.shape[1]))
    yy = y[k*batch_size : (k+1)*batch_size].reshape((batch_size,1))
    examples[k] = (Xx, yy) 

In [21]:
from multiprocessing import Pool

# Training with Hogwild!
p = Pool(5)  
p.map(mse_gradient_step, examples)

print('Loss function on the training set:', np.mean(abs(y-np.dot(X,w))))
print('Difference from the real weight vector:', abs(real_w-w).sum())

Loss function on the training set: 0.003771390334085725
Difference from the real weight vector: 0.06341289619152381
